In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *

from sklearn.ensemble import *
from datetime import datetime
from tensorflow.keras.callbacks import *

In [2]:
df = pd.read_csv("./Datasets/GW28to31/soccer-spi/spi_matches.csv")

In [3]:
df.head()

,season,date,league_id,league,team1,team2,spi1,spi2,prob1,prob2,...,importance1,importance2,score1,score2,xg1,xg2,nsxg1,nsxg2,adj_score1,adj_score2
0,2016,2016-07-09,7921,FA Women's Super League,Liverpool Women,Reading,51.56,50.42,0.4389,0.2767,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2016,2016-07-10,7921,FA Women's Super League,Arsenal Women,Notts County Ladies,46.61,54.03,0.3572,0.3608,...,NaN,NaN,2.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
2,2016,2016-07-10,7921,FA Women's Super League,Chelsea FC Women,Birmingham City,59.85,54.64,0.4799,0.2487,...,NaN,NaN,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN
3,2016,2016-07-16,7921,FA Women's Super League,Liverpool Women,Notts County Ladies,53.00,52.35,0.4289,0.2699,...,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
4,2016,2016-07-17,7921,FA Women's Super League,Chelsea FC Women,Arsenal Women,59.43,60.99,0.4124,0.3157,...,NaN,NaN,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def dateconv (dates):
  date = datetime.strptime(dates,'%Y-%m-%d') 
  return date

df['date'] = df['date'].apply(lambda x : dateconv(x))

In [5]:
last_played = datetime(2021,3,9)
GW_end = datetime(2021,4,13)
pl = df[df['league']== "Barclays Premier League"]
old = pl[pl['date']<last_played]
new = pl[pl['date']>last_played]
new = new[new['date'] < GW_end]

In [6]:
teams = pd.get_dummies(new[['team1','team2']])
teamsold = pd.get_dummies(old[['team1','team2']])

In [7]:
new = pd.concat([new,teams], axis=1)
old = pd.concat([old,teamsold], axis=1)

In [8]:
new.columns

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2', 'team1_Arsenal',
       'team1_Aston Villa', 'team1_Brighton and Hove Albion', 'team1_Burnley',
       'team1_Chelsea', 'team1_Crystal Palace', 'team1_Everton',
       'team1_Fulham', 'team1_Leeds United', 'team1_Leicester City',
       'team1_Liverpool', 'team1_Manchester City', 'team1_Manchester United',
       'team1_Newcastle', 'team1_Sheffield United', 'team1_Southampton',
       'team1_Tottenham Hotspur', 'team1_West Bromwich Albion',
       'team1_West Ham United', 'team1_Wolverhampton', 'team2_Arsenal',
       'team2_Aston Villa', 'team2_Brighton and Hove Albion', 'team2_Burnley',
       'team2_Chelsea', 'team2_Crystal Palace', 'team2_Everton',
       'team2_Fulham', 'team2_Leeds United', 'team2_Leicester City'

In [9]:
X_train = old[['season', 'spi1', 'spi2', 'score1','score2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2', 'team1_Arsenal',
       'team1_Aston Villa', 'team1_Brighton and Hove Albion', 'team1_Burnley',
       'team1_Chelsea', 'team1_Crystal Palace', 'team1_Everton',
       'team1_Fulham', 'team1_Leeds United', 'team1_Leicester City',
       'team1_Liverpool', 'team1_Manchester City', 'team1_Manchester United',
       'team1_Newcastle', 'team1_Sheffield United', 'team1_Southampton',
       'team1_Tottenham Hotspur', 'team1_West Bromwich Albion',
       'team1_West Ham United', 'team1_Wolverhampton', 'team2_Arsenal',
       'team2_Aston Villa', 'team2_Brighton and Hove Albion', 'team2_Burnley',
       'team2_Chelsea', 'team2_Crystal Palace', 'team2_Everton',
       'team2_Fulham', 'team2_Leeds United', 'team2_Leicester City',
       'team2_Liverpool', 'team2_Manchester City', 'team2_Manchester United',
       'team2_Newcastle', 'team2_Sheffield United', 'team2_Southampton',
       'team2_Tottenham Hotspur', 'team2_West Bromwich Albion',
       'team2_West Ham United', 'team2_Wolverhampton']]
X_test = new[['season', 'spi1', 'spi2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2','team1_Arsenal',
       'team1_Aston Villa', 'team1_Brighton and Hove Albion', 'team1_Burnley',
       'team1_Chelsea', 'team1_Crystal Palace', 'team1_Everton',
       'team1_Fulham', 'team1_Leeds United', 'team1_Leicester City',
       'team1_Liverpool', 'team1_Manchester City', 'team1_Manchester United',
       'team1_Newcastle', 'team1_Sheffield United', 'team1_Southampton',
       'team1_Tottenham Hotspur', 'team1_West Bromwich Albion',
       'team1_West Ham United', 'team1_Wolverhampton', 'team2_Arsenal',
       'team2_Aston Villa', 'team2_Brighton and Hove Albion', 'team2_Burnley',
       'team2_Chelsea', 'team2_Crystal Palace', 'team2_Everton',
       'team2_Fulham', 'team2_Leeds United', 'team2_Leicester City',
       'team2_Liverpool', 'team2_Manchester City', 'team2_Manchester United',
       'team2_Newcastle', 'team2_Sheffield United', 'team2_Southampton',
       'team2_Tottenham Hotspur', 'team2_West Bromwich Albion',
       'team2_West Ham United', 'team2_Wolverhampton']]

In [10]:
X_train.dropna(inplace=True)

C:\Users\Lenovo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [11]:
X_test.dropna(inplace=True)

C:\Users\Lenovo\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
lm = LinearRegression()
lm.fit(X_train.drop(['score1','score2'],axis=1),X_train[['score1','score2']])
predictions  = lm.predict(X_test)
preds = []
for num in predictions:
  score1 = np.round(num[0])
  score2 = np.round(num[1])
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Lenny Score 1'] = score1
new['Lenny Score 2'] = score2
new.to_excel("ProjectionsPLGW28to31_LR.xlsx")

36
36


In [14]:
tX_train = X_train.drop(['score1','score2'],axis=1).values
tX_test =X_test.values
y_train = X_train[['score1','score2']].values

In [15]:
scaler = MinMaxScaler()
tX_train = scaler.fit_transform(tX_train)
tX_test = scaler.transform(tX_test)
tX_train.shape

(1795, 48)

In [16]:
def two_to_three(array):
    array = np.reshape(array, array.shape+(1,))
    return array

In [17]:
tX_test = two_to_three(tX_test)
tX_train = two_to_three(tX_train)
print(tX_test.shape)
print(tX_train.shape)

(36, 48, 1)
(1795, 48, 1)


In [18]:
model = Sequential()

In [19]:

model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46))
model.add(Dense(92, activation='relu'))
model.add(Dense(2))


In [20]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [21]:
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=25)

In [22]:
model.fit(tX_train,y_train,epochs=30)

Epoch 1/30
57/57 [==============================] - 3s 23ms/step - loss: 2.0581 - accuracy: 0.5215
Epoch 2/30
57/57 [==============================] - 1s 23ms/step - loss: 1.4854 - accuracy: 0.6631
Epoch 3/30
57/57 [==============================] - 1s 22ms/step - loss: 1.4396 - accuracy: 0.6706
Epoch 4/30
57/57 [==============================] - 1s 24ms/step - loss: 1.4324 - accuracy: 0.6801
Epoch 5/30
57/57 [==============================] - 2s 27ms/step - loss: 1.3960 - accuracy: 0.6679
Epoch 6/30
57/57 [==============================] - 2s 31ms/step - loss: 1.3922 - accuracy: 0.6776
Epoch 7/30
57/57 [==============================] - 2s 31ms/step - loss: 1.2947 - accuracy: 0.7306
Epoch 8/30
57/57 [==============================] - 2s 32ms/step - loss: 1.2737 - accuracy: 0.7150
Epoch 9/30
57/57 [==============================] - 2s 31ms/step - loss: 1.3174 - accuracy: 0.6903
Epoch 10/30
57/57 [==============================] - 2s 32ms/step - loss: 1.3415 - accuracy: 0.6926
Epoch 11/

In [23]:
predictions  = model.predict(tX_test)
preds = []
for num in predictions:
  score1 = np.round(num[0]) if num[0]>=0 else 0
  score2 = np.round(num[1]) if num[1]>=0 else 0
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Tanner Score 1'] = score1
new['Tanner Score 2'] = score2
new.to_excel("ProjectionsPLGW28to31_TF.xlsx")

36
36
